In [3]:
import pretty_midi
import os
import numpy as np
import heapq
import pickle

num_notes = 128 # Number of pitches in MIDI

# We allow the network to step itself forward in time in increments of 10 ms.
# Thus the network can shift between 10 ms and 1 s, inclusive
num_time_shifts = 100 
min_time_shift = 0.01

# Represents dynamics. There are 128 possible velocities but the music transformer
# uses 32, so that's what we'll use here
num_velocities = 32

# A message can be NOTE_ON, NOTE_OFF, TIME_SHIFT, or SET_VELOCITY
message_dim = 2*num_notes + num_time_shifts + num_velocities

# quantize_velocity: takes a MIDI velocity value and puts it into the correct bin
# in our reduced representation
# ARGUMENTS
# velocity: a MIDI velocity between 0 and 127 (inclusive)
# RETURN: the quantized velocity
def quantize_velocity(velocity):
    return int(velocity*num_velocities/128)

# quantize_time_shift: takes a time shift and puts it into the correct bin
# in our reduced representation
# ARGUMENTS
# time_shift: the number of seconds to shift
# RETURN: the quantized time shift
def quantize_time_shift(time_shift):
    return int(np.round(time_shift/min_time_shift)) - 1

# note_on_event: generates the index for a NOTE_ON event
# ARGUMENTS
# note: the MIDI number for the note to be played
# RETURN: the index for a NOTE_ON message
def note_on_event(note):
    return note

# note_off_event: generates the index for a NOTE_OFF event
# ARGUMENTS
# note: the MIDI number for the note to be turned off
# RETURN: the index for the NOTE_OFF message
def note_off_event(note):
    return num_notes + note

# velocity_event: generates index for a quantized velocity
# ARGUMENTS
# velocity: a quantized MIDI velocity
# RETURN: the index of the SET_VELOCITY message
def velocity_event(velocity):
    assert(0 <= velocity and velocity < num_velocities)
    return 2*num_notes + velocity

# time_shift_event: generates the index for a TIME_SHIFT event
# ARGUMENTS
# time_shift: the quantized time shift
# RETURN: a one-hot encoding of the TIME_SHIFT message
def time_shift_event(time_shift):
    assert(0 <= time_shift and time_shift < num_time_shifts)
    return 2*num_notes + num_velocities + time_shift

# append_time_shift: appends a time shift event to the data array. If the time
# shift is too large, split it into multiple small time shifts
# ARGUMENTS
# data: time shift events will be appended to this list
# time_shift: amount of time to shift
def append_time_shift(data, time_shift):
    time_shift = quantize_time_shift(time_shift)
    
    # Split large time shifts into multiple small time shifts
    while (time_shift >= num_time_shifts):
        data.append(time_shift_event(num_time_shifts - 1))
        time_shift -= (num_time_shifts - 1)

    if (time_shift >= 0):
        data.append(time_shift_event(time_shift))
    
base_path = 'maestro-v3.0.0/'

fnum = 0 # Which file are we writing currently?

data_fnames = [] # Save file name corresponding to each numpy array

# Write to test if this becomes 4
train = 0

# Only write to a file if this is a multiple of 50
write = 0
every = 50

for year in os.listdir(base_path):
    # Skip things that aren't folders
    if year[:2] != '20':
        continue
        
    print('Starting ' + year)
    for fname in os.listdir(base_path + year):
        write += 1
        if write%every != 0:
            continue
        try:
            mid = pretty_midi.PrettyMIDI(base_path + year + '/' + fname)
        except:
            # In case of an IO error
            continue
        
        # We'll save the final data in a length L numpy array, where L is the total number of messages
        
        # First we store lists and convert to numpy arrays later
        data = []
        
        # Store the time of each message. These won't appear in the final data array
        times = []
        
        # I opened up a few maestro files, and they were all single-channel, but just to make sure, we
        # collect messages from all channels and combine them
        for i, instrument in enumerate(mid.instruments):
            # Quantized value of the last velocity message
            last_velocity = -1
            
            # Priority queue of notes to turn off and the times to turn them off.
            # Specifically, this is a list of tuples of the form (off_time, pitch),
            # where the first element of the list is always the next note to turn off
            off_queue = []
            
            time = 0
                        
            for n, note in enumerate(instrument.notes):
                # Fixes bug in 'Haydn/2104_op64n5_1.mid' where notes 674 and 675 are the same note
                if n > 0 and note.pitch == instrument.notes[n - 1].pitch and note.start == instrument.notes[n - 1].start:
                    continue
            
                # We need to turn off a note
                while off_queue and note.start > off_queue[0][0]:
                    data.append(note_off_event(off_queue[0][1]))
                    time = off_queue[0][0]
                    times.append(time)
                    heapq.heappop(off_queue)
                    
                time = note.start
                    
                velocity = quantize_velocity(note.velocity)
                if velocity != last_velocity:
                    # We have a new velocity. Add a SET_VELOCITY event
                    last_velocity = velocity
                    data.append(velocity_event(velocity))
                    times.append(time)
                
                data.append(note_on_event(note.pitch))
                times.append(time)
                
                # Add this note to the queue of notes needing to be turned off
                heapq.heappush(off_queue, (note.end, note.pitch))
                
                if n == len(instrument.notes) - 1:
                    # No more notes left. Flush the off queue
                    while off_queue:
                        data.append(note_off_event(off_queue[0][1]))
                        time = off_queue[0][0]
                        times.append(time)
                        heapq.heappop(off_queue)
            
            assert(len(times) == len(data))
        
        data = np.array(data, dtype=np.long)
        times = np.array(times, dtype=np.float)
        
        sort_idx = np.argsort(times, kind='mergesort')
        
        data = data[sort_idx]
        times = times[sort_idx]
        
        data_with_tshifts = []
        
        for m in range(data.shape[0] - 1):
            data_with_tshifts.append(data[m])
            if times[m] != times[m + 1]:
                append_time_shift(data_with_tshifts, times[m + 1] - times[m])

        data_with_tshifts = np.array(data_with_tshifts)
        
        # 80% train, 20% test
        if train == 4:
            folder = 'test'
        else:
            folder = 'train'
            
        np.save(folder + '_maestro/recording' + str(fnum) + '.npy', data_with_tshifts)
        
        train = (train + 1)%5
        
        data_fnames.append(year + '/' + fname)
        fnum += 1
        
pickle.dump(data_fnames, open( "preprocessed_data_maestro_fnames.p", "wb" ) )

Starting 2018
Starting 2011
Starting 2014
Starting 2009
Starting 2008
Starting 2004
Starting 2017
Starting 2006
Starting 2015
Starting 2013


In [1]:
import pretty_midi

In [5]:
mid = pretty_midi.PrettyMIDI('maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_16_R1_2004_01-08_ORIG_MID--AUDIO_16_R1_2004_06_Track06_wav.midi')
mid.instruments

[Instrument(program=0, is_drum=False, name="")]

674


In [255]:
print(mid.instruments[0].notes[674]) # 674 and 675 are the same note

Note(start=276.217216, end=276.309321, pitch=78, velocity=49)
